# Remote Sensing Data

This notebook works with some [Landsat 8 Collection 2 Level 2](https://planetarycomputer.microsoft.com/dataset/landsat-8-c2-l2) data, hosted by the Planetary Computer. We'll use the Planetary Computer's [STAC endpoint](https://planetarycomputer.microsoft.com/docs/reference/stac/) to query for a specific date range and area of interest, and [stackstac](https://stackstac.readthedocs.io/en/latest/) to construct an [xarray](http://xarray.pydata.org/) DataArray with all of our data.

We'll use [Dask](https://dask.org/) to distribute the computation.

In [ ]:
import numpy as np
import xarray as xr
import rioxarray

import rasterio
import stackstac
import pystac_client
import planetary_computer
import rasterio.features

import matplotlib.pyplot as plt
import cartopy.crs
import matplotlib.animation
import pandas as pd
import xrspatial.multispectral
from IPython.display import GeoJSON

from dask_gateway import GatewayCluster

In [ ]:
cluster = GatewayCluster()
client = cluster.get_client()

cluster.scale(72)
cluster

In [ ]:
aoi = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [-93.0816650390625, 40.60144147645398],
            [-91.724853515625, 40.60144147645398],
            [-91.724853515625, 41.68111756290652],
            [-93.0816650390625, 41.68111756290652],
            [-93.0816650390625, 40.60144147645398]
          ]
        ]
      }
    }
  ]
}
bbox = rasterio.features.bounds(aoi)
GeoJSON(aoi)

With `pystac_client` we can translate our desired "give me this data covering this area over this time period" into code quite naturally.

In [ ]:
%%time
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(                 # I want...
    collections=["landsat-8-c2-l2"],  # this data...
    bbox=bbox,                        # covering this area...
    datetime="2016-01-01/2020-12-31", # over this time period
    limit=500,  # batch size
)
items = list(search.items())

In [ ]:
search.matched()

In [ ]:
items[0].assets['SR_B3']

In [ ]:
items[0].properties

The actual items in blob storage aren't publicly readable (we don't want people moving all of Landsat 8 out of Azure West Europe for "free"). You just need to "sign" the assets. You can sign them anonymously, you'll just be throttled. See https://planetarycomputer.microsoft.com/docs/concepts/sas/ for more.

In [ ]:
%%time
items = pystac_client.ItemCollection([
    planetary_computer.sign_assets(item)
    for item in items
    if item.properties["eo:cloud_cover"] < 25  # percent cloudy
])

In [ ]:
%%time
ds = (
    stackstac.stack(items, assets=["SR_B2", "SR_B3", "SR_B4", "SR_B5"],
                    bounds_latlon=bbox, chunksize=2056)
        .assign_coords(band=["blue", "green", "red", "nir"])
).persist()
ds

In [ ]:
median = ds.median(dim="time").persist()
median

In [ ]:
img = xrspatial.multispectral.true_color(
    *median.isel(x=slice(2000, 3000),
                 y=slice(2000, 3000)).sel(band=["red", "green", "blue"])
).compute()

fig, ax = plt.subplots(figsize=(16, 16),
                       subplot_kw=dict(projection=cartopy.crs.AlbersEqualArea()))

image = img.plot.imshow(ax=ax, add_labels=False)

In [ ]:
ndvi = xrspatial.ndvi(median.sel(band="nir"), median.sel(band="red")).persist()
ndvi

In [ ]:
import ipyleaflet

m = ipyleaflet.Map(center=[41.01447619471347, -92.48968322606777], scroll_wheel_zoom=True)
m.layout.height = "600px"
stackstac.add_to_map(ndvi, m, name="ndvi", range=(-0.15, 0.5))
m

In [ ]:
ds

In [ ]:
red = ds.sel(band="red")
nir = ds.sel(band="nir")

ndvi_ts = (nir - red) / (nir + red)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

ndvi_ts.median(dim=["y", "x"]).plot(ax=ax);